In [1]:
import cv2
import numpy as np
import os
import imutils
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/CNN/photo mass detector/img/dataset Mn'
print("Исходный путь к рабочей директории: ", os.getcwd())
os.chdir(path)
print("Новый путь к рабочей директории: ", os.getcwd())
from google.colab.patches import cv2_imshow
from PIL import Image

Mounted at /content/drive
Исходный путь к рабочей директории:  /content
Новый путь к рабочей директории:  /content/drive/MyDrive/CNN/photo mass detector/img/dataset Mn


In [2]:
#отрисовка контуров на первичном сыром изображении
def image_contour(image):
  raw = image.copy()

  # Примените фильтр медианного размытия
  blurred_image = cv2.medianBlur(image, 5)

  # Создайте фильтр уточнения границ
  kernel = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])

  # Примените фильтр к изображению
  sharp_image = cv2.filter2D(blurred_image, -1, kernel)

  # Преобразуйте изображение в градации серого
  gray = cv2.cvtColor(sharp_image, cv2.COLOR_BGR2GRAY)

  # Создайте экземпляр фильтра CLAHE
  clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))

  # Примените фильтр CLAHE к градациям серого изображения
  contrast_enhanced = clahe.apply(gray)

  _, thresh = cv2.threshold(contrast_enhanced, 250, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)


  # Поиск контуров на бинаризованном изображении

  #contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  contours, _ = cv2.findContours(thresh, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

  return(contours, raw, contrast_enhanced)

#отрисовка контура чашки на изображении, чтобы потом по нему создать маску
def image_petri_cnt(raw, cnts_petri):

  #отрисовка незаметного контура
  raw_new = cv2.drawContours(raw, [cnts_petri], -1, (0, 1, 0), 2)
  #cv2_imshow(raw_new)
  # создаём маску по контуру чашки петри
  gray = cv2.cvtColor(raw, cv2.COLOR_RGB2GRAY)
  mask = np.zeros(gray.shape,np.uint8)
  mask = cv2.drawContours(mask, [cnts_petri], -1, 255, cv2.FILLED)
  fg_masked = cv2.bitwise_and(raw_new, raw_new, mask=mask)

  # белый фон (используем инвертированную маску)
  mask = cv2.bitwise_not(mask)
  bk = np.full(raw_new.shape, 255, dtype=np.uint8)
  bk_masked = cv2.bitwise_and(bk, bk, mask=mask)

  # сливаем фон и передний план
  final = cv2.bitwise_or(fg_masked, bk_masked)
  return(final)

#обрезает изображение по контуру чашки петри,после чего можно отрисовывать
#контуры внутри чашки(потенциально полезные)
#final - изображение, которое будет обрезаться.
#path_cutted_petri - путь, куда сохранится обрезанное изображение
def image_petri_cnt_cutter(final, path_cutted_petri):
  #код для разложения и обрезки изображения по маске контура круга чашки петри
  gray = cv2.cvtColor(final, cv2.COLOR_RGB2GRAY)
  thresh = cv2.threshold(gray, 195, 255, cv2.THRESH_BINARY)[1]
  thresh = cv2.bitwise_not(thresh)
  cnts_in_petri = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  cnts_in_petri = imutils.grab_contours(cnts_in_petri)
  cnts_in_petri = sorted(cnts_in_petri, key=cv2.contourArea, reverse=True)[:10]

  #обработка каждого контура
  i = 0
  for c in cnts_in_petri:
    if cv2.contourArea(c) > 1000:

      #рисование маски, оставляем контур
      mask = np.zeros(gray.shape, np.uint8)
      mask = cv2.drawContours(mask, [c], -1, 255, cv2.FILLED)

      #белый фон
      fg_masked = cv2.bitwise_and(raw, raw, mask=mask)
      mask = cv2.bitwise_not(mask)
      bk = np.full(raw.shape, 255, dtype=np.uint8)
      bk_masked = cv2.bitwise_and(bk, bk, mask=mask)
      finalcont = cv2.bitwise_or(fg_masked, bk_masked)

      #ограничивающая область по контуру
      output = finalcont.copy()
      x,y,w,h = cv2.boundingRect(c)

      #вырезание области с чашкой
      roi = finalcont[y:y+h, x:x+w]

      #команда для изменения размера изображения если оно будет подаваться в нейросеть
      #roi = cv2.resize(roi, (400,400))

      #сохраняем картинку
      cv2.imwrite(path_cutted_petri, roi)
      #cv2_imshow(roi)
      i += 1
      return(roi)

#выполняет стандартизацию(приводит к одному значению на разных фото)
#параметров изображения(насыщенность, якрость, контраст)
def image_standartization(img_name):
  img = cv2.imread(img_name)

  target_saturation = 200
  target_brightness = 150
  target_contrast = 80

  hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
  # Вычисление текущего уровня насыщенности, яркости и контраста
  current_saturation = hsv_img[:,:,1].mean()
  current_brightness = hsv_img[:,:,2].mean()
  current_contrast = np.std(hsv_img[:,:,2])

  # Вычисление коэффициентов для корректировки уровня насыщенности, яркости и контраста
  saturation_ratio = target_saturation / current_saturation
  brightness_ratio = target_brightness / current_brightness
  contrast_ratio = target_contrast / current_contrast

  # Применение коэффициентов к изображению
  hsv_img[:,:,1] = np.clip(hsv_img[:,:,1] * saturation_ratio, 0, 255).astype(np.uint8)
  hsv_img[:,:,2] = np.clip(hsv_img[:,:,2] * brightness_ratio, 0, 255).astype(np.uint8)

  # Преобразование изображения обратно из HSV в BGR
  corrected_img = cv2.cvtColor(hsv_img, cv2.COLOR_HSV2BGR)

  # Коррекция контраста
  lab_img = cv2.cvtColor(corrected_img, cv2.COLOR_BGR2LAB)
  l, a, b = cv2.split(lab_img)
  clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
  cl = clahe.apply(l)
  limg = cv2.merge((cl,a,b))
  final_img = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
  return final_img

#кластеризует цвета изображения, сводит их количество к минимуму для облегчения
#отделения пикселей листьев и корней
def cluster_color_cleaner(image):

  # Примените фильтр медианного размытия
  blurred_image = cv2.medianBlur(image, 3)

  #Фильтр уточнения границ изображения
  kernel = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])

  # Примените фильтр к изображению
  sharp_image = cv2.filter2D(blurred_image, -1, kernel)


  mask = np.zeros(sharp_image.shape[:2], dtype=np.uint8)
  center = (sharp_image.shape[1]//2, image.shape[0]//2)

  #обрезание чашки по ее 90 процентов объема, чтобы на изображение
  #не попали стенки чашки
  radius = int(sharp_image.shape[:2][1] * 0.9 / 2)
  cv2.circle(mask, center, radius, (255, 255, 255), -1)

  #обрезка изображения по маске
  circle_image = cv2.bitwise_and(sharp_image, sharp_image, mask=mask)

  z = circle_image.reshape((-1,3))
  z = np.float32(z)

  # define criteria, number of clusters(K) and apply kmeans()
  criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 20, 1.0)

  #4 цвета позволяют оставить на изображении только корни, фон и листья
  K = 4
  ret,label,center=cv2.kmeans(z,K,None,criteria,10, cv2.KMEANS_PP_CENTERS)

  # Convert back into uint8, and make original image
  center = np.uint8(center)
  res = center[label.flatten()]
  res2 = res.reshape((circle_image.shape))
  return(res2)

#удаляет синий фон изображения, переносит на отдельное изображение пиксели корней
#в дальнейшем необходимо, чтобы переносились пиксели листьев
def replace_blue_with_black(img):
  #img = Image.open(img)
  #pixels = img.load()
  pixels = img.copy()
  height, width, channels = img.shape

  # Наложение черных пикселей на области выше двух третей изображения(там нет корней)
  cv2.rectangle(pixels, (0, 0), (pixels.shape[1], int(height * 1/3)), (0, 0, 0), -1)

  for y in range(width - 1):
    for x in range(height - 1):

      b = pixels[x, y, 0]
      g = pixels[x, y, 1]
      r = pixels[x, y, 2]
      # если синий цвет на изображении
      # заменяем на черный цвет
      #r>=110 and r <=180 and g >=120 and g <=180 and b>=140 and b<=180:
      #100, 120,
      if r > 90 and b > 150:
        pixels[x, y] = (255, 255, 255)
      else:
        pixels[x, y] = (0, 0, 0)

  #img.save(f"Noisy_Roots_{name}.png")
  return pixels

#определяет контуры корней на изображении, переносит их в виде белых пикселей на
#черное изображение
def object_detect(img):
  #img = cv2.imread(img_name)

  #Фильтр уточнения границ изображения
  kernel = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])

  # Примените фильтр к изображению
  img = cv2.filter2D(img, -1, kernel)

  kernel = np.ones((3, 3), np.uint8)
  img = cv2.dilate(img, kernel, iterations=1)

  #конвертация изображения в серый
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  _, thresh = cv2.threshold(gray, 250, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

  # Поиск контуров на бинаризованном изображении
  min_contour_area = 400
  contours, _ = cv2.findContours(thresh, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

  #удаление белых областей(шум соль) с изображения
  for contour in contours:
    if cv2.contourArea(contour) < min_contour_area:
      cv2.drawContours(img, [contour], 0, (0, 0, 0), -1)

  #cv2.imwrite(f"Cleaned_{img_name}.png", img)
  return img

#извлекает все имена файлов в папке
def filenames_finder(path):
  filenames = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
  return(filenames)

#функция разархивации для датасетов
#archive_format = "zip"
def archive_extractor(path, extract_dir, archive_format):
  import shutil
  shutil.unpack_archive(path, extract_dir, archive_format)
  print("Archive file unpacked successfully.")

#вычленяет с изображения зеленые пикселию
def leaves_detect(img):
  #img = Image.open(img_name)
  #pixels = img.load()
  pixels = img.copy()
  height, width, channels = img.shape

  mask  = np.zeros((height, width), dtype=np.uint8)
  mask[int(height*0.6):, :] = 255
  pixels[mask == 255] = [0, 0, 0]

  for y in range(width - 1):
    for x in range(height - 1):

      b = pixels[x, y, 0]
      g = pixels[x, y, 1]
      r = pixels[x, y, 2]

      if g > b and g > r and b < 100:
        pixels[x, y] = (255, 255, 255)
      else:
        pixels[x, y] = (0, 0, 0)

  #img.save(f"{output_path}Noisy_Leaves_{name}.png")
  return pixels

In [ ]:
#переменная пути, где хранятся сырые данные - необрезанные чашки петри
path_dataset = '/content/drive/MyDrive/magic/dataset 01.02/2/'

#переменная пути, где хранятся обрезанные чашки петри
roi_path_dataset = '/content/drive/MyDrive/magic/dataset 01.02/2'

#создание списка имен изображений в папке
filenames_Mn = filenames_finder(path_dataset)

#смена рабочей папки на ту, куда будут сохранятся изображения листьев на черном фоне
os.chdir("/content/drive/MyDrive/CNN/photo mass detector/img/leaves/")

#удаление имени объекта .ipynb из списка с именами изображений
filenames_Mn.pop(-1)

#цикл перебирает каждое исходное изображение, обрезает до чашки, сохраняет в новую папку
i = 0
for el in filenames_Mn:
  print(el)

  test1 = cv2.imread(f'/content/drive/MyDrive/magic/dataset 01.02/2/{el}')
  contours, raw, contrast_enhanced = image_contour(test1)

  #путь сохранения обрезанного изображения чашки петри с растениями(roi)
  path_cutted_petri = f"/content/drive/MyDrive/CNN/photo mass detector/img/roi_petri/TEST_{i}.jpg"

  #сортировка контуров в изначальном изображении, чтобы найти контур чашки
  cnts_petri = sorted(contours, key=cv2.contourArea, reverse=True)[1]

  #удаление всей внешней области фотографии, не входящей в контур петри
  final = image_petri_cnt(raw, cnts_petri)

  #обрезка изображения по контуру чашки петри
  roi =  image_petri_cnt_cutter(final, f'/content/drive/MyDrive/CNN/photo mass detector/img/roi_petri/Petri_cutted_{i}.jpg')
  i += 1

In [ ]:
#путь с папкой сырых изображений
path_roi = '/content/drive/MyDrive/CNN/photo mass detector/img/roi_petri'

#путь для сохранения стандартизированных изображений
path_standartised = '/content/drive/MyDrive/CNN/photo mass detector/img/petri standartised/'

os.chdir(path_roi)

#поиск имен всех файлов в папке
filenames = filenames_finder(path_roi)

#цикл перебирает все обрезанные чашки петри и приводит параметры фото к
#одинаковым значениям контраста, яркости и т.п.
i = 0
for filename in filenames:
  standartised_petri = image_standartization(filename)
  output_path = os.path.join(path_standartised, f'standartised_{i}.png' )
  cv2.imwrite(output_path, standartised_petri)
  i += 1

#поиск имен стандартизированных изображений в новой папке
path_roi = '/content/drive/MyDrive/CNN/photo mass detector/img/petri standartised'
os.chdir(path_roi)
filenames = filenames_finder(path_roi)

#цикл перебирает стандартизированные изображения, сводя количество цветов на нем
#к 4 основным(фон, корни, листья)
for filename in filenames:
  outname = f"/content/drive/MyDrive/CNN/photo mass detector/img/clustered petri/{filename}"
  cluster_color_cleaner(filename, outname)

#поиск имен кластеризованных изображений в новой папке
path_clustered_color = '/content/drive/MyDrive/CNN/photo mass detector/img/clustered petri/'
#os.chdir(path_clustered_color)
filenames = filenames_finder(path_clustered_color)
path_noisy_roots = '/content/drive/MyDrive/CNN/photo mass detector/img/noisy roots'
os.chdir(path_noisy_roots)

#путь, куда сохранять картинки с листьями
leaves_path = '/content/drive/MyDrive/CNN/photo mass detector/img/noisy_leaves/'
#цикл перебирает кластеризованные изображения чтобы разделить на отдельные
#изображения 1- корни, 2 - листья
i = 0
for filename in filenames:
  filename = os.path.join(path_clustered_color, filename)

  #функция детекции корней
  replace_blue_with_black(filename, i)

  #функция детекции листьев
  leaves_detect(filename, i, leaves_path)
  i += 1

#смена рабочей директории на ту, где хранятся отделенные корни
path_clean_roots = '/content/drive/MyDrive/CNN/photo mass detector/img/cleaned roots'
os.chdir(path_clean_roots)

#получение списка имен изображений в папке корней
filenames = filenames_finder(path_noisy_roots)

#очистка изображения корня от артефактов и перенос на новое изображение только
#картинки корней
i = 0
for filename in filenames:

  filename = os.path.join(path_noisy_roots, filename)
  print(filename)
  object_detect(filename, i)
  i += 1

#смена рабочей директории на ту, где хранятся отделенные листья
path_clean_roots = '/content/drive/MyDrive/CNN/photo mass detector/img/cleaned roots'
os.chdir(path_clean_roots)

#получение списка имен изображений в папке корней
filenames = filenames_finder(leaves_path)

#очистка изображения корня от артефактов и перенос на новое изображение только
#картинки корней

path_clean_leaves = '/content/drive/MyDrive/CNN/photo mass detector/img/cleaned leaves'
os.chdir(path_clean_leaves)

i = 0
for filename in filenames:

  filename = os.path.join(leaves_path, filename)
  print(filename)
  object_detect(filename, i)
  i += 1


In [ ]:
#путь с папкой сырых изображений
path_roi = '/content/drive/MyDrive/CNN/photo mass detector/img/roi_petri'

#путь для сохранения стандартизированных изображений
path_standartised = '/content/drive/MyDrive/CNN/photo mass detector/img/petri standartised/'

path_noisy_roots = '/content/drive/MyDrive/CNN/photo mass detector/img/noisy roots'

path_noisy_leaves = '/content/drive/MyDrive/CNN/photo mass detector/img/noisy leaves'

path_clean_roots = '/content/drive/MyDrive/CNN/photo mass detector/img/cleaned roots/'

path_clean_leaves = '/content/drive/MyDrive/CNN/photo mass detector/img/cleaned leaves/'

#поиск имен всех файлов в папке
filenames = filenames_finder(path_roi)
print(filenames)
#цикл перебирает все обрезанные чашки петри и приводит параметры фото к
#одинаковым значениям контраста, яркости и т.п.
i = 0
for filename in filenames:
  path_roi_file = os.path.join(path_roi, filename)
  standartised_petri = image_standartization(path_roi_file)
  clustered_img = cluster_color_cleaner(standartised_petri)

  root_img = clustered_img.copy()
  leaves_img = clustered_img.copy()

  #корни
  noisy_roots_img = replace_blue_with_black(root_img)
  #noisy_roots_img.save(f"{path_noisy_roots}Noisy_roots_{i}.png")
  cleaned_roots_img = object_detect(noisy_roots_img)
  path_roots_final = os.path.join(path_clean_roots, f"clean_roots_{i}.png")
  cv2.imwrite(path_roots_final, cleaned_roots_img)

  #листья
  noisy_leaves_img = leaves_detect(leaves_img)
  #noisy_leaves_img.save(f"{path_noisy_leaves}Noisy_leaves_{i}.png")
  cleaned_leaves_img = object_detect(noisy_leaves_img)
  path_leaves_final = os.path.join(path_clean_leaves, f"clean_leaves_{i}.png")
  cv2.imwrite(path_leaves_final, cleaned_leaves_img)


  #output_path = os.path.join(path_standartised, f'standartised_{i}.png' )
  #cv2.imwrite(output_path, standartised_petri)
  i += 1


['Petri_cutted_1.jpg', 'Petri_cutted_2.jpg', 'Petri_cutted_3.jpg', 'Petri_cutted_4.jpg', 'Petri_cutted_5.jpg', 'Petri_cutted_6.jpg', 'Petri_cutted_7.jpg', 'Petri_cutted_8.jpg', 'Petri_cutted_0.jpg']


'\n#поиск имен стандартизированных изображений в новой папке\npath_roi = \'/content/drive/MyDrive/CNN/photo mass detector/img/petri standartised\'\nos.chdir(path_roi)\nfilenames = filenames_finder(path_roi)\n\n#цикл перебирает стандартизированные изображения, сводя количество цветов на нем\n#к 4 основным(фон, корни, листья)\nfor filename in filenames:\n  outname = f"/content/drive/MyDrive/CNN/photo mass detector/img/clustered petri/{filename}"\n  cluster_color_cleaner(filename, outname)\n\n#поиск имен кластеризованных изображений в новой папке\npath_clustered_color = \'/content/drive/MyDrive/CNN/photo mass detector/img/clustered petri/\'\n#os.chdir(path_clustered_color)\nfilenames = filenames_finder(path_clustered_color)\npath_noisy_roots = \'/content/drive/MyDrive/CNN/photo mass detector/img/noisy roots\'\nos.chdir(path_noisy_roots)\n\n#путь, куда сохранять картинки с листьями\nleaves_path = \'/content/drive/MyDrive/CNN/photo mass detector/img/noisy_leaves/\'\n#цикл перебирает кластер